In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/project

/content/drive/My Drive/project


In [0]:
!ls 

'Copy of gender_with_file_xml.ipynb'   split_xml.pl
 data				       tnt.rar
 data_test.csv			       tnt.xml
 finalized_model.sav		       Untitled0.ipynb
 gender.ipynb			       Untitled1.ipynb
 gender_with_file_xml.ipynb	       Untitled2.ipynb
 parser.ipynb			       vk_part_k0.xml
 script_xml.pl


In [0]:
from lxml import etree
import numpy as np
import pandas as pd
import re

In [0]:
#Находит значение атрибута
def parse_attributes(attribute, line):
    result = re.search(r'{}=.(\w+).'.format(attribute), line)
    if result:
        return result.group(1)
    #Если информации нет, возвращает строку 'NA'
    return 'NA'

#Применяет ругулярку к строке
def parse_text(reg, line):
    result = re.search(reg, line)
    if result:
        return result.group(1)
    #<o composite="true"> не обробатывается (я не знаю что это) и еще что то непонятное
    #МОЖЕТ ВЕРНУТЬ ПУСТУЮ СТРОКУ
    return ""
        
def parse_article(file):
    df = pd.DataFrame()
    
    attributes_list = ["source", "genrei", "gender", "birth", "nickname",
                      "id", "loc", "month", "rule", "year"]
    
    attributes_dict = {}
    for attr in attributes_list:
        attributes_dict[attr] = []
    attributes_dict["text"] = []
    attributes_dict["words"] = []
    attributes_dict["infinitives"] = []
    attributes_dict["specification"] = []
    
    text = ""
    words = []
    infinitives = []
    specification = []
    
    for line in file:
        if line.startswith('</text>'):
            
            attributes_dict["text"].append(text)
            attributes_dict["words"].append(words)
            attributes_dict["infinitives"].append(infinitives)
            attributes_dict["specification"].append(specification)
            
            words = []
            specification = []
            infinitives = []
            text = ""
            
        elif line.startswith('<text'):
            
            for attr in attributes_list:
                if attr == "loc":
                    attributes_dict[attr].append(re.findall(r'loc=.(\w+).', line))
                else:
                    attributes_dict[attr].append(parse_attributes(attr, line))
        else:
            
            words.append(parse_text(r'(\S+)\s+\S+\s+\S+', line))
            infinitives.append(parse_text(r'\S+\s+\S+\s+(\S+)', line))
            specification.append(parse_text(r'\S+\s+(\S+)\s+\S+', line))
            text += line + "\n"
            
    df = df.from_dict(attributes_dict)
    return df

In [0]:
def parse_file(file):
    with open(file) as file:
        return parse_article(file)

In [0]:
def find_F_M(form):
  if form == "": 
    return '', ''
  if (form[0] == 'N' or form[0] == 'M' or form[0] == 'Y') and len(form) > 2 :
    return form[0], form[2]
  if form [0] == 'V' and len(form) > 6:
    return form[0], form[6]
  if (form[0] == 'A' or form[0] == 'P') and len(form) > 3:
    return form[0], form[3] 
  else:
    return '', ''

In [0]:
import json
keyword_F = {}
keyword_M = {}
with open('./data/data_json/keyword.json', 'r') as f:
    dict_vk = json.load(f)
    keyword_F = dict_vk["keyword_F"]
    keyword_M = dict_vk["keyword_M"]

In [0]:
def has_i(form):
  if len(form) < 5:
    return False
  if form[0] == 'P' and form[2] == '1' and form[4] == 's' and form[5] == 'n':
    return True
  else:
    return False

In [0]:
def to_num(dt):
  X_oh = pd.DataFrame(columns=['keyness_F', 'keyness_M', 'N_F', 'N_M', 'V_F', 'V_M', 'A_F', 'A_M', 'P_F', 'P_M', 'M_F', 'M_M', 'Y_F', 'Y_M', 'YA'])
  X1 = dt['infinitives']
  X2 = dt['specification']

  for sentence, forms in zip(X1, X2):
    F = 0.
    M = 0.

    N_F = 0.
    N_M = 0.

    V_F = 0.
    V_M = 0.

    A_F = 0.
    A_M = 0.

    P_F = 0.
    P_M = 0.

    M_F = 0.
    M_M = 0.

    Y_F = 0.
    Y_M = 0.

    has = 0.
    for word, form in zip(sentence, forms):
      if word == 'я' and has_i(form):
        has = 1.
      if keyword_F.get(word) != None:
        a = keyword_F[word]
      else:
        a = 0.
      F = F + a
      if keyword_M.get(word) != None:
        b = keyword_M[word]
      else:
        b = 0.
      M = M + b

      (first, gender) = find_F_M(form)

      if first == 'N':
        if gender == 'm':
          N_M += 1.
        if gender == 'f':
          N_F += 1.

      if first == 'V':
        add = 1.
        if form[4] == '1':
          add = 1000.
        if gender == 'm':
          V_M += add
        if gender == 'f':
          V_F += add

      if first == 'A':
        if gender == 'm':
          A_M += 1.
        if gender == 'f':
          A_F += 1.


      if first == 'P':
        if gender == 'm':
          P_M += 1.
        if gender == 'f':
          P_F += 1.

      if first == 'M':
        if gender == 'm':
          M_M += 1.
        if gender == 'f':
          M_F += 1.

      if first == 'Y':
        if gender == 'm':
          Y_M += 1.
        if gender == 'f':
          Y_F += 1.
    X_oh = X_oh.append({'keyness_F': F, 'keyness_M': M, 'N_F': N_F, 'N_M':N_M, 'V_F':V_F, 'V_M':V_M, 'A_F':A_F, 'A_M':A_M, 'P_F':P_F, 'P_M':P_M, 'M_F':M_F, 'M_M':M_M, 'Y_F':Y_F, 'Y_M':Y_M, 'YA':has}, ignore_index=True)
  return X_oh

In [0]:
def keyword(keyness, c):
  kw = {}
  for key in keyness.keys():
    if keyness[key] > c:
      kw.update({key: keyness[key]})
  return kw

keyword_F_new = keyword(keyword_F, 1.02)
keyword_M_new = keyword(keyword_M, 1.02)

pos = 0
keywords = {}
for key in keyword_F_new.keys():
  key = key.strip('"({[]})[&?<>/\:;,*^!@#+-=._%$0123457689')
  if keywords.get(key) == None and key[:4] != "http":
    keywords.update({key : pos})
    pos = pos + 1
    
for key in keyword_M_new.keys():
  key = key.strip('"({[]})[&?<>/\:;,*^!@#+-=._%$0123457689')
  if keywords.get(key) == None and key[:4] != "http":
    keywords.update({key : pos})
    pos = pos + 1
kw = [*keywords]

def by_keywords(data):
  X_by_kw = np.array([0] * (data.shape[0]*len(kw))).reshape(data.shape[0], len(kw))
  X = data['infinitives']
  n = 0
  for sentence in X:
    for word in sentence:
      if keywords.get(word) != None:
        X_by_kw[n][keywords[word]] += 1
    n += 1
  return X_by_kw

In [0]:
def gender_to_num(y):
  x = []
  for yi in y:
    if yi == "F":
      x.append(0)
    else:
      x.append(1)
  return x

In [0]:
for k in range(1, 48):
  print(k)
  file_in = './data/data_xml/vk_part_k{}.xml'.format(k)
  file_out = './data/data_num/vk_part_k{}'.format(k)
  data = parse_file(file_in)
  column = ['gender', 'infinitives', 'specification']
  data = data[column]
  data_F = data[data.gender == "F"]
  data_M = data[data.gender == "M"]
  sh = list(range(data_F.shape[0]))
  np.random.shuffle(sh)
  index = sh[:data_M.shape[0]]
  data_F = data_F.iloc[index]
  dt = data_F.append(data_M, ignore_index=True)
  X_by_kw = by_keywords(dt)
  X_oh = to_num(dt)
  y = dt['gender']
  y = np.array(gender_to_num(y))
  X_new = np.hstack((X_by_kw, X_oh))
  X_new = np.vstack((X_new.transpose(), y)).transpose()

  np.save(file_out, X_new)
  del data, dt, data_F, data_M, X_oh, X_new, X_by_kw

31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
